# Topic Modeling

## Imports de llibreries

In [232]:
import pandas as pd
import numpy as np
import sklearn
from stop_words import get_stop_words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import ne_chunk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
import spacy

## Càrrega del dataset

In [233]:
df_proposals = pd.read_csv("../data/processed/processed_proposals.csv")

## Text de les propostes

Primer carreguem el dataset de stopwords en català perquè no es tinguin en compte a l'hora de fer topic modeling i després tokenitzem el text havent-lo lemmatitzat abans per passar les paraules a singular, masculí, etc... per evitar problemes.

In [237]:

nlp = spacy.load("ca_core_news_sm")

stop_words = set(nltk.corpus.stopwords.words('catalan'))

def clean_text(headline):
    # Tokenize the text
    doc = nlp(headline)
    
    # Lemmatize and filter tokens
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.text.lower() not in stop_words and len(token.text) > 3]
    
    # Join the tokens back into a string
    cleaned_text = " ".join(tokens)
    return cleaned_text

df_proposals['body_preprocessed'] = df_proposals['body/ca'].apply(clean_text)
df_proposals.head()

,id,title/ca,body/ca,endorsements/total_count,comments,attachments,followers,published_at,is_amend,published_at_dies,body_preprocessed
0,87446,Participar Assemblea Ciutadana pel Clima de Ca...,"Buenos días, me gustaría participar en la asam...",4,0,0,2,2023-10-09 11:27:07+00:00,False,123.937350,Buenos días gustaríar participar asamblea ciud...
1,87447,Assemblea ciutadana pel clima de Catalunya,M'agradaria participar en aquesta assemblea pe...,1,1,0,3,2023-10-10 08:38:57+00:00,False,123.054132,agradar participar assemblea clima rebar carta...
2,87452,"Transició a producció, comerç i consum ecològi...","La producció, comerç i consum d'aliments no ec...",1,0,0,2,2023-10-13 10:32:35+00:00,False,119.975220,producció comerç consum aliment ecològic local...
3,87453,"L'aigua, el principal aliment.",La major part dels pous catalans ni són legals...,3,0,0,3,2023-10-13 10:40:31+00:00,False,119.969711,major part pou català legal monitoratge contin...
4,87454,Prohibir construccions no bioclimàtiques,La normativa no hauria de permetre cap constru...,3,0,0,3,2023-10-13 10:50:31+00:00,False,119.962766,normativa haver permetre construcció edificaci...


### Model LSA (Latent Semantic Analysis o TruncatedSVD)

In [238]:
vect = TfidfVectorizer(max_features=59, stop_words=list(stop_words)) # Tocar max_features per adaptar el model
vect_text = vect.fit_transform(df_proposals['body_preprocessed'])

# Paràmetres a modificar són n_components i n_iter
lsa_model = TruncatedSVD(n_components=59, algorithm='randomized',n_iter=60, random_state=42)
lsa_top = lsa_model.fit_transform(vect_text)


# most important words for each topic
vocab = vect.get_feature_names_out()

In [239]:
def get_top_words_for_document(doc_index, lsa_model, vocab, num_top_words=7): # Modificar num_top_words per mostrar quantes top words considerem
    doc_topic_distribution = lsa_model.transform(vect_text[doc_index])
    
    top_topic_index = np.argmax(doc_topic_distribution)
    
    top_words_indices = lsa_model.components_[top_topic_index].argsort()[:-num_top_words-1:-1]
    top_words = [vocab[j] for j in top_words_indices]
    
    return top_words

df_proposals['top_topic_words_lsa'] = df_proposals.index.map(lambda x: get_top_words_for_document(x, lsa_model, vocab))


In [240]:
df_proposals.head(10)

,id,title/ca,body/ca,endorsements/total_count,comments,attachments,followers,published_at,is_amend,published_at_dies,body_preprocessed,top_topic_words_lsa
0,87446,Participar Assemblea Ciutadana pel Clima de Ca...,"Buenos días, me gustaría participar en la asam...",4,0,0,2,2023-10-09 11:27:07+00:00,False,123.937350,Buenos días gustaríar participar asamblea ciud...,"[participar, clima, agradar, assemblea, tema, ..."
1,87447,Assemblea ciutadana pel clima de Catalunya,M'agradaria participar en aquesta assemblea pe...,1,1,0,3,2023-10-10 08:38:57+00:00,False,123.054132,agradar participar assemblea clima rebar carta...,"[participar, clima, agradar, assemblea, tema, ..."
2,87452,"Transició a producció, comerç i consum ecològi...","La producció, comerç i consum d'aliments no ec...",1,0,0,2,2023-10-13 10:32:35+00:00,False,119.975220,producció comerç consum aliment ecològic local...,"[ciutadà, parar, local, pagès, territori, cald..."
3,87453,"L'aigua, el principal aliment.",La major part dels pous catalans ni són legals...,3,0,0,3,2023-10-13 10:40:31+00:00,False,119.969711,major part pou català legal monitoratge contin...,"[aigua, participar, energia, transport, agrada..."
4,87454,Prohibir construccions no bioclimàtiques,La normativa no hauria de permetre cap constru...,3,0,0,3,2023-10-13 10:50:31+00:00,False,119.962766,normativa haver permetre construcció edificaci...,"[nou, projecte, procés, crear, faltar, gent, a..."
5,87458,Projectes del futur,"Hola, sóc un jove graduat en enginyeria; M'agr...",1,0,0,1,2023-10-14 14:38:54+00:00,False,118.804167,Hola jove graduat enginyeria agradar participa...,"[aigua, participar, energia, transport, agrada..."
6,87460,Ordenació medioambiental. Tractament dels resi...,1.- Residus: Es podria instal.lar màquines de ...,5,0,0,2,2023-10-14 14:58:27+00:00,False,118.790590,Residus poder instal.lar màquina recollida lla...,"[aigua, participar, energia, transport, agrada..."
7,87461,Procés de selecció,"Referent al procés de selecció, no entenc perq...",3,3,0,1,2023-10-15 16:24:45+00:00,False,117.730660,referent procés selecció entendre envíen aleat...,"[participar, clima, agradar, assemblea, tema, ..."
8,87462,"Decreixement, Aigua i Sobirania alimentària ag...",Estem entrant en un caos climàtic terrible. Ai...,5,0,0,4,2023-10-15 19:02:35+00:00,False,117.621053,entrar caos climàtic terrible Aigua Sobirania ...,"[elèctric, ciutadà, procés, alimentari, sobira..."
9,87464,Apropament entre generació i consum d'energia ...,Crec que caldria potenciar l'apropament entre ...,1,1,0,2,2023-10-15 20:58:48+00:00,False,117.540347,creure caldre potenciar apropament producció c...,"[caldre, elèctric, energia, solar, actualment,..."


Evidentment, no és un model perfecte i existeixen propostes que no es corresponen del tot amb el seu topic número 1, però en general la majoria de textos de les propostes tenen una bona relació amb els tòpics detectats pel Latent Semantic Analysis model.

### LDA Model (Latent Dirichlet Allocation)

In [259]:
lda_model = LatentDirichletAllocation(n_components=59, learning_method='online', random_state=42, max_iter=60)
lda_top = lda_model.fit_transform(vect_text)

def get_top_words_for_document_lda(doc_index, lda_model, vect, num_top_words=7):
    doc_topic_distribution = lda_model.transform(vect_text[doc_index])
    
    top_topic_index = np.argmax(doc_topic_distribution)
    
    top_words_indices = lda_model.components_[top_topic_index].argsort()[:-num_top_words-1:-1]
    top_words = [vocab[j] for j in top_words_indices]
    
    return top_words

df_proposals['top_topic_words_lda'] = df_proposals.index.map(lambda x: get_top_words_for_document_lda(x, lda_model, vocab))

In [261]:
df_proposals.head(30)

,id,title/ca,body/ca,endorsements/total_count,comments,attachments,followers,published_at,is_amend,published_at_dies,body_preprocessed,top_topic_words_lsa,top_topic_words_lda
0,87446,Participar Assemblea Ciutadana pel Clima de Ca...,"Buenos días, me gustaría participar en la asam...",4,0,0,2,2023-10-09 11:27:07+00:00,False,123.937350,Buenos días gustaríar participar asamblea ciud...,"[participar, clima, agradar, assemblea, tema, ...","[parar, tema, clima, participar, placa, ciutad..."
1,87447,Assemblea ciutadana pel clima de Catalunya,M'agradaria participar en aquesta assemblea pe...,1,1,0,3,2023-10-10 08:38:57+00:00,False,123.054132,agradar participar assemblea clima rebar carta...,"[participar, clima, agradar, assemblea, tema, ...","[participar, agradar, assemblea, clima, projec..."
2,87452,"Transició a producció, comerç i consum ecològi...","La producció, comerç i consum d'aliments no ec...",1,0,0,2,2023-10-13 10:32:35+00:00,False,119.975220,producció comerç consum aliment ecològic local...,"[ciutadà, parar, local, pagès, territori, cald...","[territori, local, consum, aliment, producció,..."
3,87453,"L'aigua, el principal aliment.",La major part dels pous catalans ni són legals...,3,0,0,3,2023-10-13 10:40:31+00:00,False,119.969711,major part pou català legal monitoratge contin...,"[aigua, participar, energia, transport, agrada...","[residu, alimentari, públic, sostenible, part,..."
4,87454,Prohibir construccions no bioclimàtiques,La normativa no hauria de permetre cap constru...,3,0,0,3,2023-10-13 10:50:31+00:00,False,119.962766,normativa haver permetre construcció edificaci...,"[nou, projecte, procés, crear, faltar, gent, a...","[nou, trajecte, ex, crear, elèctric, servei, t..."
5,87458,Projectes del futur,"Hola, sóc un jove graduat en enginyeria; M'agr...",1,0,0,1,2023-10-14 14:38:54+00:00,False,118.804167,Hola jove graduat enginyeria agradar participa...,"[aigua, participar, energia, transport, agrada...","[participar, agradar, assemblea, clima, projec..."
6,87460,Ordenació medioambiental. Tractament dels resi...,1.- Residus: Es podria instal.lar màquines de ...,5,0,0,2,2023-10-14 14:58:27+00:00,False,118.790590,Residus poder instal.lar màquina recollida lla...,"[aigua, participar, energia, transport, agrada...","[instal, davant, aigua, solar, solució, agrada..."
7,87461,Procés de selecció,"Referent al procés de selecció, no entenc perq...",3,3,0,1,2023-10-15 16:24:45+00:00,False,117.730660,referent procés selecció entendre envíen aleat...,"[participar, clima, agradar, assemblea, tema, ...","[procés, participar, gent, faltar, canvi, part..."
8,87462,"Decreixement, Aigua i Sobirania alimentària ag...",Estem entrant en un caos climàtic terrible. Ai...,5,0,0,4,2023-10-15 19:02:35+00:00,False,117.621053,entrar caos climàtic terrible Aigua Sobirania ...,"[elèctric, ciutadà, procés, alimentari, sobira...","[aigua, aire, consum, actualment, àmbit, procé..."
9,87464,Apropament entre generació i consum d'energia ...,Crec que caldria potenciar l'apropament entre ...,1,1,0,2,2023-10-15 20:58:48+00:00,False,117.540347,creure caldre potenciar apropament producció c...,"[caldre, elèctric, energia, solar, actualment,...","[energia, transport, producció, sostenible, pl..."
